## In this notebook we will identify route pricing by quarter

In [34]:
import pandas as pd

In [35]:
# Read in top 1000 DF - WE WILL USE THIS DF TO OBTAIN PRICING
top1000 = pd.read_csv('./data/raw/Consumer_Airfare_Report__Table_1_-_Top_1_000_Contiguous_State_City-Pair_Markets.csv')
print(top1000.shape)
top1000.head()

(95023, 18)


,Year,quarter,citymarketid_1,citymarketid_2,city1,city2,nsmiles,passengers,fare,carrier_lg,large_ms,fare_lg,carrier_low,lf_ms,fare_low,table_1_flag,Geocoded_City1,Geocoded_City2
0,2009,2,32467,34576,"Miami, FL (Metropolitan Area)","Rochester, NY",1204,203,151.46,FL,0.29,131.05,FL,0.29,131.05,1,"Miami, FL (Metropolitan Area)\n(44.977479, -93...","Rochester, NY\n(43.155708, -77.612547)"
1,2000,4,30397,33198,"Atlanta, GA (Metropolitan Area)","Kansas City, MO",692,782,172.83,DL,0.63,194.71,NJ,0.26,126.88,1,"Atlanta, GA (Metropolitan Area)\n(33.748547, -...","Kansas City, MO\n(39.099792, -94.578559)"
2,2007,4,32575,34614,"Los Angeles, CA (Metropolitan Area)","Salt Lake City, UT",590,3122,135.24,DL,0.51,144.28,B6,0.15,111.68,1,"Los Angeles, CA (Metropolitan Area)\n(34.05223...","Salt Lake City, UT\n(40.758478, -111.888142)"
3,2004,4,32337,31650,"Indianapolis, IN","Minneapolis/St. Paul, MN",503,395,206.78,NW,0.74,224.77,TZ,0.11,156.74,1,"Indianapolis, IN\n(39.76845, -86.156212)","Minneapolis/St. Paul, MN\n(44.977479, -93.264346)"
4,2008,4,30194,30559,"Dallas/Fort Worth, TX","Seattle, WA",1670,957,242.74,AA,0.47,262.43,AS,0.27,218.90,1,"Dallas/Fort Worth, TX\n(40.11086, -77.035636)","Seattle, WA\n(47.603229, -122.33028)"


### Column Creation

In [36]:
# CREATE - Route Column
top1000['market_city'] = top1000['city1'] + ' - ' + top1000['city2']

# CREATE - Datetime Column
monthly = top1000['Year'].astype(str) + 'M' + (3 * top1000['quarter']).astype(str)
from statsmodels.tsa.base.datetools import dates_from_str
monthly = dates_from_str(monthly)
top1000['year-month'] = pd.DatetimeIndex(monthly)

top1000.head()

,Year,quarter,citymarketid_1,citymarketid_2,city1,city2,nsmiles,passengers,fare,carrier_lg,large_ms,fare_lg,carrier_low,lf_ms,fare_low,table_1_flag,Geocoded_City1,Geocoded_City2,market_city,year-month
0,2009,2,32467,34576,"Miami, FL (Metropolitan Area)","Rochester, NY",1204,203,151.46,FL,0.29,131.05,FL,0.29,131.05,1,"Miami, FL (Metropolitan Area)\n(44.977479, -93...","Rochester, NY\n(43.155708, -77.612547)","Miami, FL (Metropolitan Area) - Rochester, NY",2009-06-30
1,2000,4,30397,33198,"Atlanta, GA (Metropolitan Area)","Kansas City, MO",692,782,172.83,DL,0.63,194.71,NJ,0.26,126.88,1,"Atlanta, GA (Metropolitan Area)\n(33.748547, -...","Kansas City, MO\n(39.099792, -94.578559)","Atlanta, GA (Metropolitan Area) - Kansas City, MO",2000-12-31
2,2007,4,32575,34614,"Los Angeles, CA (Metropolitan Area)","Salt Lake City, UT",590,3122,135.24,DL,0.51,144.28,B6,0.15,111.68,1,"Los Angeles, CA (Metropolitan Area)\n(34.05223...","Salt Lake City, UT\n(40.758478, -111.888142)","Los Angeles, CA (Metropolitan Area) - Salt Lak...",2007-12-31
3,2004,4,32337,31650,"Indianapolis, IN","Minneapolis/St. Paul, MN",503,395,206.78,NW,0.74,224.77,TZ,0.11,156.74,1,"Indianapolis, IN\n(39.76845, -86.156212)","Minneapolis/St. Paul, MN\n(44.977479, -93.264346)","Indianapolis, IN - Minneapolis/St. Paul, MN",2004-12-31
4,2008,4,30194,30559,"Dallas/Fort Worth, TX","Seattle, WA",1670,957,242.74,AA,0.47,262.43,AS,0.27,218.90,1,"Dallas/Fort Worth, TX\n(40.11086, -77.035636)","Seattle, WA\n(47.603229, -122.33028)","Dallas/Fort Worth, TX - Seattle, WA",2008-12-31


In [37]:
top1000 = top1000[['year-month', 'market_city', 'city1', 'city2', 'fare']]
print(top1000.shape)
top1000.head()

(95023, 5)


,year-month,market_city,city1,city2,fare
0,2009-06-30,"Miami, FL (Metropolitan Area) - Rochester, NY","Miami, FL (Metropolitan Area)","Rochester, NY",151.46
1,2000-12-31,"Atlanta, GA (Metropolitan Area) - Kansas City, MO","Atlanta, GA (Metropolitan Area)","Kansas City, MO",172.83
2,2007-12-31,"Los Angeles, CA (Metropolitan Area) - Salt Lak...","Los Angeles, CA (Metropolitan Area)","Salt Lake City, UT",135.24
3,2004-12-31,"Indianapolis, IN - Minneapolis/St. Paul, MN","Indianapolis, IN","Minneapolis/St. Paul, MN",206.78
4,2008-12-31,"Dallas/Fort Worth, TX - Seattle, WA","Dallas/Fort Worth, TX","Seattle, WA",242.74


In [38]:
top1000['year-month'] = pd.to_datetime(top1000['year-month'])
top1000 = top1000.set_index('year-month').sort_index()
print(top1000.shape)
top1000.head()

(95023, 4)


,market_city,city1,city2,fare
year-month,,,,
1996-03-31,"Cleveland, OH (Metropolitan Area) - Denver, CO","Cleveland, OH (Metropolitan Area)","Denver, CO",234.76
1996-03-31,"Denver, CO - Minneapolis/St. Paul, MN","Denver, CO","Minneapolis/St. Paul, MN",120.28
1996-03-31,"Las Vegas, NV - Phoenix, AZ","Las Vegas, NV","Phoenix, AZ",73.30
1996-03-31,"Atlanta, GA (Metropolitan Area) - Buffalo, NY","Atlanta, GA (Metropolitan Area)","Buffalo, NY",199.34
1996-03-31,"Los Angeles, CA (Metropolitan Area) - San Anto...","Los Angeles, CA (Metropolitan Area)","San Antonio, TX",162.77


In [39]:
top1000 = top1000.groupby(['market_city', 'city1', 'city2', pd.Grouper(freq='M')])[['fare']].sum().reset_index().set_index('year-month').sort_index()
print(top1000.shape)
top1000.head()

(95023, 4)


,market_city,city1,city2,fare
year-month,,,,
1996-03-31,"Hartford, CT - West Palm Beach/Palm Beach, FL","Hartford, CT","West Palm Beach/Palm Beach, FL",129.20
1996-03-31,"Minneapolis/St. Paul, MN - San Francisco, CA (...","Minneapolis/St. Paul, MN","San Francisco, CA (Metropolitan Area)",290.73
1996-03-31,"Cincinnati, OH - Tampa, FL (Metropolitan Area)","Cincinnati, OH","Tampa, FL (Metropolitan Area)",153.17
1996-03-31,"Denver, CO - Portland, OR","Denver, CO","Portland, OR",240.01
1996-03-31,"Los Angeles, CA (Metropolitan Area) - Phoenix, AZ","Los Angeles, CA (Metropolitan Area)","Phoenix, AZ",73.67


In [40]:
# test data 
top1000[top1000['market_city'] == 'Hartford, CT - West Palm Beach/Palm Beach, FL']

,market_city,city1,city2,fare
year-month,,,,
1996-03-31,"Hartford, CT - West Palm Beach/Palm Beach, FL","Hartford, CT","West Palm Beach/Palm Beach, FL",129.20
1996-06-30,"Hartford, CT - West Palm Beach/Palm Beach, FL","Hartford, CT","West Palm Beach/Palm Beach, FL",132.35
1996-09-30,"Hartford, CT - West Palm Beach/Palm Beach, FL","Hartford, CT","West Palm Beach/Palm Beach, FL",118.88
1996-12-31,"Hartford, CT - West Palm Beach/Palm Beach, FL","Hartford, CT","West Palm Beach/Palm Beach, FL",132.74
1997-03-31,"Hartford, CT - West Palm Beach/Palm Beach, FL","Hartford, CT","West Palm Beach/Palm Beach, FL",130.03
...,...,...,...,...
2018-09-30,"Hartford, CT - West Palm Beach/Palm Beach, FL","Hartford, CT","West Palm Beach/Palm Beach, FL",178.87
2018-12-31,"Hartford, CT - West Palm Beach/Palm Beach, FL","Hartford, CT","West Palm Beach/Palm Beach, FL",229.45
2019-03-31,"Hartford, CT - West Palm Beach/Palm Beach, FL","Hartford, CT","West Palm Beach/Palm Beach, FL",202.92


In [46]:
# top1000['market_city'].value_counts() == 95

In [50]:
top1000.to_csv('./data/clean/routepricing_byquarter.csv')